In [1]:
import matplotlib as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.layers import Dense,Flatten,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import cv2 as cv

In [2]:
img_h,img_w=180,180

In [3]:
batch_size=30
data_dir="Combined_dataset"

In [4]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    label_mode='binary',
    image_size=(img_h,img_w),
    batch_size=batch_size
)

Found 3432 files belonging to 3 classes.
Using 2574 files for training.


In [5]:
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    label_mode='binary',
    image_size=(img_h,img_w),
    batch_size=batch_size
)

Found 3432 files belonging to 3 classes.
Using 858 files for validation.


# Model Implementation

In [6]:
resnet_model=Sequential()
pretrained_model=tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(180,180,3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)
#freezing all convolutional layers 
for layer in pretrained_model.layers:
  layer.trainable=False

#unfreezing last convolutional layer block
for layer in pretrained_model.layers[-11:]:
  layer.trainable=True

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512,activation='relu'))
resnet_model.add(Dense(3,activation='softmax'))

pretrained_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 180, 180, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 186, 186, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 90, 90, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 90, 90, 64)   256         ['conv1_conv[0][0]']      

Resnet layers Summary

In [ ]:
import pandas as pd
layers = [(layer.name, layer.trainable) for layer in pretrained_model.layers]
df = pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable'])
pd.set_option('display.max_rows', None)
print(df)

Compiling Model

In [8]:
from tensorflow.keras.metrics import Precision, Recall
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy', metrics=['accuracy',Precision(),Recall()])

In [9]:
epochs=20
checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_iris.h5', verbose=1, save_best_only=True)

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs')]
history =resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks
)

Layer ModuleWrapper was created by passing
non-serializable argument values in `__init__()`,
and therefore the layer must override `get_config()` in
order to be serializable. Please implement `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2, **kwargs):
        super().__init__(**kwargs)
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Epoch 1/20
86/86 [==============================] - 179s 2s/step - loss: 0.6921 - accuracy: 0.6892 - precision: 0.7024 - recall: 0.6686 - val_loss: 0.8702 - val_accuracy: 0.6725 - val_precision: 0.6733 - val_recall: 0.6725
Epoch 2/20
86/86 [==============================] - 189s 2s/step - loss: 0.3860 - accuracy: 0.8213 - precision: 0.8279 - recall: 0.8131 - val_loss: 0.7300 - val_accuracy: 0.7692 - val_precision: 0.7699 - val_recall: 0.7681
Epoch 3/20
86/86 [==============================] - 191s 2s/step - loss: 0.2683 - accuracy: 0.8823 - precision: 0.8858 - recall: 0.8772 - val_loss: 1.2799 - val_accuracy: 0.7517 - val_precision: 0.7515 - val_recall: 0.7506
Epoch 4/20
86/86 [==============================] - 182s 2s/step - loss: 0.1762 - accuracy: 0.9277 - precision: 0.9278 - recall: 0.9239 - val_loss: 0.6725 - val_accuracy: 0.8275 - val_precision: 0.8273 - val_recall: 0.8263
Epoch 5/20
86/86 [==============================] - 170s 2s/step - loss: 0.0958 - accuracy: 0.9662 - precisi

In [10]:
img_dir = "D:\Iris Images\IIITD_Contact_Lens_Iris_DB\Vista Scanner\Colored"
import os
img_files = os.listdir(img_dir)
features_list = []

In [37]:
def max_of_pairs(lst):
    print(lst.shape)
    return [max(lst[i:i+4]) for i in range(0, len(lst), 4)]

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
for img_file in img_files:
    # Load the image and preprocess it
    img_path = os.path.join(img_dir, img_file)
    image=cv2.imread(img_path)
    image_resized=cv2.resize(image,(img_h,img_w))
    image=np.expand_dims(image_resized,axis=0)
    features=resnet_model.layers[-3].output
    feature=keras.models.Model(inputs=resnet_model.input,outputs=features).predict(image)
    feature = max_of_pairs(feature)
    features_list.append(feature)
# feature=keras.models.Model(inputs=resnet_model.input, outputs=features).predict(image)
# print(feature.shape)


In [ ]:
# write a for loop to change the features_list[i] dimension to 1x512 using max_of_pairs function
for i in range(len(features_list)):
    features_list[i] = max_of_pairs(features_list[i])

print(len(features_list))

In [28]:
features_array = np.vstack(features_list)
df = pd.DataFrame(features_array)
num_features = features_array.shape[1]
col_names = ['feature{}'.format(i) for i in range(1, num_features+1)]
df.columns = col_names
df['filename'] = img_files
df.set_index('filename', inplace=True)
excel_file = "D:\\Libraries(Python)\\OpenCV\\vista_colored.xlsx"
df.to_excel(excel_file)

Exception ignored in: <function ZipFile.__del__ at 0x000001E1668555A0>
Traceback (most recent call last):
  File "c:\Program Files\Python310\lib\zipfile.py", line 1819, in __del__
    self.close()
  File "c:\Program Files\Python310\lib\zipfile.py", line 1836, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [34]:
print(features_array.shape)

(1150, 2048)
